In [15]:
import pandas as pd
pd.options.display.max_columns = 99
chunk_iter = pd.read_csv('/Users/teitelbaumsair/Desktop/Open-Source-Projects/Analyzing 2018 Fundraising Startup Deals/crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

Compute each column's missing value counts


In [16]:
mv_list = []
for chunk in chunk_iter:
    mv_list.append(chunk.isnull().sum())
    
combined_mv_vc = pd.concat(mv_list)
unique_combined_mv_vc = combined_mv_vc.groupby(combined_mv_vc.index).sum()
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [17]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
counter = 0
series_memory_fp = pd.Series(dtype='float64')
for chunk in chunk_iter:
    if counter == 0:
        series_memory_fp = chunk.memory_usage(deep=True)
    else:
        series_memory_fp += chunk.memory_usage(deep=True)
    counter += 1

# Drop memory footprint calculation for the index.
series_memory_fp = series_memory_fp.drop('Index')
series_memory_fp

company_permalink         3634836
company_name              3167958
company_category_code     3003288
company_country_code      2749224
company_state_code        2687027
company_region            2988593
company_city              3087190
investor_permalink        4557604
investor_name             3492202
investor_category_code     602880
investor_country_code     2320340
investor_state_code       2188119
investor_region           2973185
investor_city             2561163
funding_round_type        2987771
funded_at                 3119249
funded_month              2960648
funded_quarter            2960648
funded_year                422960
raised_amount_usd          422960
dtype: int64

In [18]:
series_memory_fp.sum() / (1024 * 1024)


np.float64(50.43777942657471)

In [19]:
unique_combined_mv_vc.sort_values()


company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [20]:
# Drop columns representing URLs or containing too many missing values (>90% missing)
drop_cols = ['investor_permalink', 'company_permalink', 'investor_category_code']
keep_cols = chunk.columns.drop(drop_cols)

In [21]:
keep_cols.tolist


<bound method IndexOpsMixin.tolist of Index(['company_name', 'company_category_code', 'company_country_code',
       'company_state_code', 'company_region', 'company_city', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_type', 'funded_at', 'funded_month',
       'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')>

### Selecting Data Types

In [22]:
# Key: Column name, Value: List of types
col_types = {}
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

for chunk in chunk_iter:
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))

In [23]:
uniq_col_types = {}
for k,v in col_types.items():
    uniq_col_types[k] = set(col_types[k])
uniq_col_types

{'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_name': {'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_state_code': {'float64', 'object'},
 'investor_region': {'object'},
 'investor_city': {'float64', 'object'},
 'funding_round_type': {'object'},
 'funded_at': {'object'},
 'funded_month': {'object'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'raised_amount_usd': {'float64'}}

In [24]:
chunk

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,Mortimer Singer,NaN,NaN,unknown,NaN,series-a,2012-10-01,2012-10,2012-Q4,2012,3060000.0
50001,ChaCha,advertising,USA,IN,Indianapolis,Carmel,Morton Meyerson,NaN,NaN,unknown,NaN,series-b,2007-10-01,2007-10,2007-Q4,2007,12000000.0
50002,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2008-04-18,2008-04,2008-Q2,2008,500000.0
50003,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,750000.0
50004,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,Mr. Andrew Oung,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52865,Garantia Data,enterprise,USA,CA,SF Bay,Santa Clara,Zohar Gilon,NaN,NaN,unknown,NaN,series-a,2012-08-08,2012-08,2012-Q3,2012,3800000.0
52866,DudaMobile,mobile,USA,CA,SF Bay,Palo Alto,Zohar Gilon,NaN,NaN,unknown,NaN,series-c+,2013-04-08,2013-04,2013-Q2,2013,10300000.0
52867,SiteBrains,software,USA,CA,SF Bay,San Francisco,zohar israel,NaN,NaN,unknown,NaN,angel,2010-08-01,2010-08,2010-Q3,2010,350000.0
52868,Comprehend Systems,enterprise,USA,CA,SF Bay,Palo Alto,Zorba Lieberman,NaN,NaN,unknown,NaN,series-a,2013-07-11,2013-07,2013-Q3,2013,8400000.0


### Load Data to PostgreSQL

In [26]:
import pandas as pd
from sqlalchemy import create_engine

# Database credentials
db_name = "CBData"
db_user = "postgres"
db_password = "1234"
db_host = "localhost"  # Change if hosted elsewhere
db_port = "5433"  # Default PostgreSQL port

# Create a connection to PostgreSQL
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Load CSV in chunks and insert into PostgreSQL
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

for chunk in chunk_iter:
    chunk.to_sql("investments", engine, if_exists='append', index=False)

print("Data successfully loaded into PostgreSQL.")

DataError: (psycopg2.errors.InvalidDatetimeFormat) invalid input syntax for type date: "2012-10"
LINE 1: ...'New York', 'New York', 'series-a', '2012-10-30', '2012-10',...
                                                             ^

[SQL: INSERT INTO investments (company_permalink, company_name, company_category_code, company_country_code, company_state_code, company_region, company_city, investor_permalink, investor_name, investor_category_code, investor_country_code, investor_state_ ... 535836 characters truncated ... s, %(funded_month__999)s, %(funded_quarter__999)s, %(funded_year__999)s, %(raised_amount_usd__999)s)]
[parameters: {'investor_name__0': '1-800-FLOWERS.COM', 'raised_amount_usd__0': 2000000.0, 'company_state_code__0': 'CA', 'company_permalink__0': '/company/advercar', 'investor_category_code__0': None, 'funded_year__0': 2012, 'company_country_code__0': 'USA', 'funded_month__0': '2012-10', 'company_city__0': 'San Francisco', 'investor_city__0': 'New York', 'investor_co